In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from simility_requests.return_rule_info_from_simility import ReturnRuleInfoFromSimility
from simility_apis.set_password import set_password
import pandas as pd
import numpy as np
import json

In [4]:
# set_password()

# params = {
#     'url': 'https://eu.simility.com',
#     'app_prefix': 'omnyex',
#     'user': 'jlaidler@paypal.com',
#     'entity': 'orders'
# }

# rri = ReturnRuleInfoFromSimility(**params)

# rule_info = rri.request()

# pd.DataFrame(rule_info['jsonConfig'].apply(lambda x: x['conditions'])).to_csv('omnyex_rule_conditions.csv', )

In [5]:
# conditions = pd.read_csv('rw_rule_conditions.csv', index_col=[0])
# conditions = pd.read_csv('omnyex_rule_conditions.csv', index_col=[0])
conditions = pd.read_json('quoka_rules.json'); conditions.set_index('name', inplace=True); conditions = pd.DataFrame(conditions['conditions']); conditions.columns = ['jsonConfig']

In [6]:
# conditions

In [7]:
conditions_dicts = conditions['jsonConfig'].apply(lambda x: json.loads(x))

In [14]:
for rn, r in conditions_dicts.items():
    if "not_in" in str(r):
        print(r)

{'condition': 'AND', 'rules': [{'id': 'ad_price_type', 'field': 'ad_price_type', 'type': 'string', 'operator': 'not_in', 'value': 'value.FREE,NEGOTIATION'}]}


In [80]:
from rule_parsing.from_system_parser import FromSystemParser

In [81]:
rule_dicts = {}
for i, (rule_name, condition_dict) in enumerate(conditions_dicts.iteritems()):
    print(i, end='\r')
    try:
        sp = FromSystemParser(conditions_dict=condition_dict)
        rule_dicts[rule_name] = sp.convert()
    except:
        rule_dicts[rule_name] = None

In [82]:
len(rule_dicts.keys()), len(conditions_dicts.keys())

(175, 175)

In [83]:
'''
37, 53, 138, 153
'''
'''
Omnyex: 3, 4, 14, 15, 113, 135, 163, 167, 168
'''
conditions_dicts[49]

{'condition': 'OR',
 'rules': [{'id': 'java_el',
   'field': 'java_el',
   'type': 'string',
   'input': 'text',
   'operator': 'java_el',
   'value': '${table_values.$shipping_address_2.toLowerCase().matches(".*(aim)\\\\b.*") }'},
  {'id': 'java_el',
   'field': 'java_el',
   'type': 'string',
   'input': 'text',
   'operator': 'java_el',
   'value': '${table_values.$shipping_address_1.toLowerCase().matches(".*(3501|12208 ky highway 36 w)\\\\b.*") }'},
  {'id': 'shipping_address_1.ln_shipping_address_1_escalate',
   'field': 'shipping_address_1.ln_shipping_address_1_escalate',
   'type': 'integer',
   'operator': 'equal',
   'value': 'value.1'},
  {'id': 'shipping_address_2.ln_shipping_address_2_escalate',
   'field': 'shipping_address_2.ln_shipping_address_2_escalate',
   'type': 'integer',
   'operator': 'equal',
   'value': 'value.1'},
  {'id': 'shipping_address_full.ln_shipping_address_full_escalate',
   'field': 'shipping_address_full.ln_shipping_address_full_escalate',
   'type'

In [84]:
list(rule_dicts.values())[49]

# Test dict to string class

In [85]:
from rule.rule import ConvertRuleDictToString

In [86]:
rule_strings = {}
for i, (rule_name, rule_dict) in enumerate(rule_dicts.items()):
    print(i, end='\r')
    if rule_dict is None:
        rule_strings[rule_name] = None
    else:
        converter = ConvertRuleDictToString(rule_dict)
        rule_strings[rule_name] = converter.convert(as_numpy=True)

In [87]:
len(rule_strings)

175

In [88]:
list(rule_dicts.values())[6]

{'condition': 'AND',
 'rules': [{'field': 'billing_address_full_num_distinct_full_name_per_billing_address_full_1day',
   'operator': 'greater_or_equal',
   'value': 1.0},
  {'field': 'billing_address_full_num_distinct_full_name_per_billing_address_full_7day',
   'operator': 'greater_or_equal',
   'value': 1.0},
  {'field': 'billing_address_full_num_distinct_full_name_per_billing_address_full_30day',
   'operator': 'greater_or_equal',
   'value': 3.0}]}

In [89]:
list(rule_strings.values())[6]

"(X['billing_address_full_num_distinct_full_name_per_billing_address_full_1day'].values>=1.0)&(X['billing_address_full_num_distinct_full_name_per_billing_address_full_7day'].values>=1.0)&(X['billing_address_full_num_distinct_full_name_per_billing_address_full_30day'].values>=3.0)"

# Test dict to lambda class

In [90]:
from rule.rule import ConvertRuleDictToLambda

In [91]:
rule_lambdas = {}
for i, (rule_name, rule_dict) in enumerate(rule_dicts.items()):
    print(i, end='\r')
    if rule_dict is None:
        rule_lambdas[rule_name] = None
    else:
        converter = ConvertRuleDictToLambda(rule_dict)
        rule_lambdas[rule_name] = converter.convert(as_numpy=True, with_kwargs=False)

In [92]:
# rule_lambda_list[0]({'payer_id_sum_approved_txn_amt_per_paypalid_1day': 60, 'payer_id_sum_approved_txn_amt_per_paypalid_7day': 120, 'payer_id_sum_approved_txn_amt_per_paypalid_30day': 500, 'num_items': 1, 'total_num_items_ordered':2})
list(rule_lambdas.values())[6](60, 120, 500, 1, 2, 6, 7)

"(X['billing_address_full_num_distinct_full_name_per_billing_address_full_1day'].values>=60)&(X['billing_address_full_num_distinct_full_name_per_billing_address_full_7day'].values>=120)&(X['billing_address_full_num_distinct_full_name_per_billing_address_full_30day'].values>=500)"

# Test string to dict

In [93]:
from rule.rule import ConvertRuleStringToDict

In [97]:
# test = "((np.char.contains(X['message_text_1'].values, ' web de '))|(np.char.contains(X['message_text_1'].values, ' ( ] '))|(np.char.contains(X['message_text_1'].values, ' .at. '))|(np.char.contains(X['message_text_1'].values, ' [ ] '))|(np.char.contains(X['message_text_1'].values, ' (a) '))|(np.char.contains(X['message_text_1'].values, ' att '))|(np.char.contains(X['message_text_1'].values, ' ett '))|(np.char.contains(X['message_text_1'].values, ' .et. '))|(np.char.contains(X['message_text_1'].values, ' -at- '))|(np.char.contains(X['message_text_1'].values, ' (et) '))|(np.char.contains(X['message_text_1'].values, ' -et- '))|(np.char.contains(X['message_text_1'].values, 'liksmail'))|(np.char.contains(X['message_text_1'].values, 'posteo.net'))|(np.char.contains(X['message_text_1'].values, '[at]'))|(np.char.contains(X['message_text_1'].values, '[punkt]')))&((X['user_sender_email_domain'].values=='jeepmails.com')|(X['user_sender_email_domain'].values=='outlook.com')|(X['user_sender_email_domain'].values=='tutamail.com')|(X['user_sender_email_domain'].values=='web.de')|(X['user_sender_email_domain'].values=='gmx.de')|(X['user_sender_email_domain'].values=='gmx.at')|(X['user_sender_email_domain'].values=='gmx.ch')|(X['user_sender_email_domain'].values=='gmx.net'))&((~np.char.startswith(X['user_sender_id_eid'].values, '1'))&(~np.char.startswith(X['user_sender_id_eid'].values, '20'))&(~np.char.startswith(X['user_sender_id_eid'].values, '21'))&(~np.char.startswith(X['user_sender_id_eid'].values, '22')))&((~np.char.contains(X['message_text_1'].values, 'gesendet mit der web.de mail app')))"


In [98]:
# c = ConvertRuleStringToDict(rule_string = test)
# c.convert()

In [99]:
rule_dicts_from_strings = {}
rule_dicts_equal = {}
for i, (rule_name, rule_string) in enumerate(rule_strings.items()):
#     print(i, end='\r')
    print(rule_name, end='\r')
    if rule_string is None:
        rule_dicts_from_strings[rule_name] = None
    else:
        c = ConvertRuleStringToDict(rule_string = rule_string)
        rule_dict_from_str = c.convert()
        rule_dicts_from_strings[rule_name] = rule_dict_from_str
        rule_dicts_equal[rule_name] = rule_dict_from_str == rule_dicts[rule_name]

In [100]:
np.array(list(rule_dicts_equal.values())).mean()

0.9681528662420382

In [101]:
rule_names_dicts_not_equal = []
for rule_name, equal in rule_dicts_equal.items():
    rule_dict_str = str(rule_dicts[rule_name])
    rule_dict_from_string_str = str(rule_dicts_from_strings[rule_name])
    if not equal and rule_dict_str.count("'contains'") > 1 and rule_dict_from_string_str.count("'contains'") > 1:
        rule_names_dicts_not_equal.append(rule_name)

In [102]:
rule_names_dicts_not_equal

[]

In [104]:
# rule_strings['LoanCreditFraud_new']

In [105]:
# rule_dicts['LoanCreditFraud_new']

In [106]:
# rule_dicts_from_strings['LoanCreditFraud_new']

# Test dict to string (from dicts converted using string to dict)

In [107]:
rule_strings_from_dicts = {}
rule_strings_equal = {}
for rule_name, rule_dict in rule_dicts_from_strings.items():
    print(rule_name, end='\r')
    if rule_dict is None:
        rule_strings_from_dicts[rule_name] = None
    else:
        c = ConvertRuleDictToString(rule_dict=rule_dict)
        rule_str = c.convert(as_numpy=True)
        rule_strings_from_dicts[rule_name] = rule_str
        rule_strings_equal[rule_name] = rule_str == rule_strings[rule_name]
        

In [108]:
np.array(list(rule_strings_equal.values())).mean()

0.9808917197452229

In [109]:
rule_strings_not_equal = []
for rule_name, equal in rule_strings_equal.items():
    if not equal:
        rule_strings_not_equal.append(rule_name)

In [110]:
rule_strings_not_equal

['Rule_Suggester_1553604152209',
 'Rule_Suggester_1553604164562',
 'Rule_Suggester_1553604168748']

In [114]:
rule_strings['Rule_Suggester_1553604164562']

"((X['submitter_full_name_num_distinct_account_number_per_full_name_30day'].values>=4.0))&((X['account_number_variability_shipping_zip_to_account_number_1day'].values>=0.4))"

In [113]:
rule_strings_from_dicts['Rule_Suggester_1553604164562']

"(X['submitter_full_name_num_distinct_account_number_per_full_name_30day'].values>=4.0)&(X['account_number_variability_shipping_zip_to_account_number_1day'].values>=0.4)"

In [25]:
from rules.rules import Rules
from rules.convert_rule_string_to_rule_dict import ConvertRuleStringToRuleDict

In [26]:
rule_strings = {
        'Rule1': "((X['A']>=60.0)|(X['B'].str.startswith('foo', na=False))|(X['C'].isna()))&(X['D'].isin(['foo', 'bar']))&(X['E']==X['F'])&(X['G'].fillna('')=='')",
        'Rule2': "((X['A']<=60.0)|(~X['B'].str.startswith('foo', na=False))|(~X['C'].isna()))&(~X['D'].isin(['foo', 'bar']))&(X['E']!=X['F'])&(X['G'].fillna('')!='')"
    }

In [30]:
r = Rules(rule_strings=rule_strings)

In [32]:
r.system_dicts

{}

In [33]:
# r._rule_strings_to_rule_dicts()
r.as_rule_dicts()

{}

In [29]:
r.rule_dicts

{}

In [7]:
converter = ConvertRuleStringToRuleDict(rule_string=rule_strings['Rule1'])

In [8]:
converter.convert()

{'condition': 'AND',
 'rules': [{'condition': 'OR',
   'rules': [{'field': 'A', 'operator': 'greater_or_equal', 'value': 60.0},
    {'field': 'B', 'operator': 'begins_with', 'value': 'foo'},
    {'field': 'C', 'operator': 'is_null', 'value': None}]},
  {'field': 'D', 'operator': 'in', 'value': ['foo', 'bar']},
  {'field': 'E', 'operator': 'equal_field', 'value': 'F'},
  {'field': 'G', 'operator': 'is_empty', 'value': None}]}